In [ ]:
! apt-get install poppler-utils
! sudo apt install tesseract-ocr
! sudo apt-get install tesseract-ocr
! pip install pdf2image
! pip install pytesseract
! wget "https://nss.org.sg/report/1eebac90-8Avi_2005_19_2.pdf" # DONE
! wget "https://nss.org.sg/report/8baffd7a-9Avi_2005_19_1.pdf" # DONE
! wget "https://nss.org.sg/report/481b0c11-eAvi_2005_19_3.pdf" # DONE
! wget "https://nss.org.sg/report/3d428c84-fAvi_2006_19_4.pdf" # DONE
! pip install word2number
% load_ext google.colab.data_table

import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')
nltk.download('maxent_treebank_pos_tagger') 

from google.colab import drive
import pandas as pd

drive.mount('/content/gdrive')
df_in = pd.read_csv("/content/gdrive/My Drive/df (10).csv")
datafile = pd.read_csv("/content/gdrive/My Drive/ebird-checklist.csv", encoding= 'unicode_escape')

In [ ]:
from pdf2image import convert_from_path, convert_from_bytes
import cv2
import pytesseract
import numpy as np
import re
from google.colab.patches import cv2_imshow
from tqdm.notebook import tqdm
from functools import partial

tqdm = partial(tqdm, position=0, leave=True)

images = convert_from_bytes(open('3d428c84-fAvi_2006_19_4.pdf', 'rb').read())

text = ''

for i in tqdm(images[4:30]):
  open_cv_image = np.array(i) 
  img = open_cv_image[:, :, ::-1].copy()
  img = cv2.resize(img, None, fx=1.5, fy=1.5, interpolation=cv2.INTER_CUBIC)

  gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
  # gray = cv2.medianBlur(gray, 3)
  gray, img_bin = cv2.threshold(gray,128,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)

  gray = cv2.bitwise_not(img_bin)

  kernel = np.ones((2, 1), np.uint8)
  img = cv2.erode(gray, kernel, iterations=1)
  img = cv2.dilate(img, kernel, iterations=1)
  tt = pytesseract.image_to_string(cv2.bitwise_not(img))
  text += (tt.replace("/\n","/") + "\n\n")

In [ ]:
import pprint
from nltk.tokenize import word_tokenize
from nltk import pos_tag, ne_chunk
import pandas as pd
import nltk.tag, nltk.data
import copy
from word2number import w2n

res = re.compile(r'^([A-Z -]){5,}.*?[A-Z]?[a-z ]*$')
splitter = re.compile(r'([({]a?l?l?[A-Z/ ]*[)/}]|\(sev obs\))')
date = re.compile(r'[0-9]{1,2} ?/ ?[0-9]{1,2}')

class Sighting:
  def __init__(self, species, date, count, person, location, string=None):
    self.person = person
    self.date = date
    self.species = species
    self.count = count
    self.string = string
    self.location = location
    self.data = [species, date, count, person, location, string]
  def __str__(self):
    return f"SPECIES: {self.species}\nDATE: {self.date}\nCOUNT: {self.count}\nPERSON: {self.person}\nLOCATION: {self.location}\nCOMMENTS: {self.string}"
  def join(self, other):
    data2 = zip(self.data, other.data)
    datanew = []
    n0 = 0
    for a, b in data2:
      if n0 == 3:
        n0 += 1
        if a in ['', 0] and b not in ['', 0]:
          datanew.append([b, b])
        continue
      if a in ['', 0] and b not in ['', 0]:
        datanew.append([b, b])
      elif a not in ['', 0] and b in ['', 0]:
        datanew.append([a, a])
      else:
        datanew.append([a, b])
      n0 += 1
    return (Sighting(*[a for a, b in datanew]), Sighting(*[b for a, b in datanew]))

def make_df(sightings):
  d = {
      "Species": [i.species for i in sightings],
      "Date": [i.date for i in sightings],
      "Count": [i.count for i in sightings],
      "Observer": [i.person for i in sightings],
      "Location": [i.location for i in sightings],
      #"Comments": [i.string for i in sightings]
  }
  return pd.DataFrame.from_dict(d)

spdata = {}
spdata2 = []

latest = ''

for t in text.split("\n"):
  if re.match(res, t)!=None:
    if t not in spdata.keys():
      spdata[t] = ''
    latest = t
  elif latest != '':
    spdata[latest] += (' ' + t)

n = 0

for k, i in list(spdata.items()):
  n += 1
  if len(date.findall(i)) <= len(splitter.findall(i)):
    s = splitter.split(i)
    dates = (date.findall(i))
    li = list(zip(s[::2], s[1::2], dates))
    li = [(a.replace(c,''), b, c) for a, b, c in li ]
  else:
    s = date.split(i)
    dates = (date.findall(i))
    obs = splitter.findall(i)
    li = list(zip(s[::2], obs, dates))
    li = [(a.replace(c,''), b, c) for a, b, c in li ]
  for string, person, d in li:
    original = copy.deepcopy(string)
    string = string.rstrip(', ').lstrip(', ')
    for repl in ["A pair", "a pair", "1 pair", "pair", "Pair", "1 male and 1 female"]:
      string = string.replace(repl, "2")
    string = string.replace("Singapore Avifauna", '')
    for repl in ["a juvenile", "a female", "a male", "a summer male", "another", "\|", "an immature", '\[']:
      string = re.compile(repl, re.IGNORECASE).sub('1', string)
    if all([word in ["and", ",", ''] for word in string.split(" ")]) or len(string) == 0:
      spdata2.append(Sighting(k, d, 0, person, '', string=string))
      continue
    location = []
    count = []
    wtk = word_tokenize(string)
    pos_tag1 = pos_tag(wtk)
    chunk = ne_chunk(pos_tag1)
    for ix in chunk.subtrees(filter=lambda x: x.label() in ["PERSON", "ORGANIZATION", "FACILITY", "LOCATION"]):
      location.append(" ".join([a for a, b in ix]))
    if len(location) == 0:
      for ix in wtk:
        if ix[0].isupper():
          location.append(ix)
      location = [" ".join(location)]
    location = [l for l in location if l not in ["Sungei Puaka", "Kg Melayu", "Nesting Report", "Ficus", "Trail", "African Tulip Tree"]]
    for ix in chunk:
      if len(ix) > 1 and (ix[1] == "CD") and '/' not in ix[0]:
        count.append(ix[0])
    if "Lane 2" in string:
      count.remove('2')
    string = re.compile("[0-9]+ ?meters", re.IGNORECASE).sub('', string)
    string = re.compile("[0-9]+ ?metres", re.IGNORECASE).sub('', string)
    if len(count) != 1:
      while True:
        inp = input(f"COUNT for {string.strip()}      {n}/{len(spdata.items())}\n").strip()
        if inp == 'skip': 
          break
        try: 
          if inp == '':
            count = 0
            break
          count1 = int(inp)
          break
        except:
          if inp == 'more':
            print(k, i)
          continue
    else:
      inp = count[0].replace('.', '')
    if count != 0 and inp != 'skip': 
      try:
        count = int(inp)
      except ValueError:
        try:
          count = w2n.word_to_num(inp)
        except ValueError:
          while True:
            inp = input(f"COUNT for {string.strip()}      {n}/{len(spdata.items())}\n").strip()
            if inp == 'skip': 
              break
            try: 
              if inp == '':
                count = 0
                break
              count1 = int(inp)
              break
            except:
              if inp == 'more':
                print(k, i)
              continue
    if len(location) > 1:
      while True:
        inp = input(f"LOCATION for {string.strip()}; options: {location}      {n}/{len(spdata.items())}\n").strip()
        if inp == 'skip': 
          break
        try: 
          if inp == '':
            location = ''
            break
          location1 = location[int(inp)]
          break
        except:
          if inp == 'more':
            print(k, i)
          continue
    else:
      if inp != 'skip':
        inp = 0
    try:
      if location != '' and inp != 'skip':
        location = location[int(inp)]
    except ValueError:
      location = " ".join([ location[i] for i in map(int, inp.split(',')) ])
    if inp != 'skip':
      spdata2.append(Sighting(k, d, count, person, location, string=string))
    else:
      print("skipped this one")

for x in range(1, len(spdata2)):
  spdata2[x], spdata2[x-1] = spdata2[x].join(spdata2[x-1])

In [164]:
from google.colab import files
from datetime import datetime
import editdistance
from tqdm.notebook import tqdm
from IPython.display import display, HTML, clear_output
import time
import pickle
from google.colab import data_table

tqdm.pandas()

# cachedeng = {k: list(datafile.loc[datafile["scientific name"] == v]["English name"])[0] for k, v in cachedsci.items()}
# print(cachedeng)

# a_file = open("cachedeng.pkl", "wb")
# pickle.dump(cachedeng, a_file)
# a_file.close()

overrides = {
    "Lonchura molucca": "Lonchura atricapilla",
    "Black-faced Munia": "Black-headed Munia",
    "Nilgiri Flowerpecker": "Plain Flowerpecker",
    "Dicaeum concolor": "Dicaeum minullum",
    "Collared Scops-Owl": "Sunda Scops-Owl",
    "Otus lettia": "Otus lempiji"
}

cachedsci = pickle.load(open("cachedsci.pkl", "rb"))
cachedeng = pickle.load(open("cachedeng.pkl", "rb"))
cachedsci = {k: v if v not in overrides.keys() else overrides[v] for (k, v) in cachedsci.items()}
cachedeng = {k: v if v not in overrides.keys() else overrides[v] for (k, v) in cachedeng.items()}

done = 0

def get(spname):
  global done
  datafile2 = datafile.copy(deep=True)
  text = spname.lower()
  datafile2["Distance"] = datafile2["Pair"].apply(lambda i: editdistance.eval(text, i))
  datafile2 = datafile2.sort_values(by='Distance')
  vsci = list(datafile2["scientific name"])[0]
  veng = list(datafile2["English name"])[0]
  score = list(datafile2["Distance"])[0]
  if score > 5:
    try:
      print(text, done)
      n = 3
      display(datafile2.head(3))
      inpu = input("Confirm?").strip()
      while inpu == 'more':
        n += 5
        display(datafile2.head(n))
        inpu = input("Confirm?").strip()
      if inpu != '':
        vsci = list(datafile2["scientific name"])[int(inpu)]
        veng = list(datafile2["English name"])[int(inpu)]
        inpu2 = input("Confirm again").strip()
        if inpu != '':
          vsci = list(datafile2["scientific name"])[int(inpu2)]
          veng = list(datafile2["English name"])[int(inpu2)]
    except ValueError:
      text2 = input("Species name").strip()
      datafile2["Distance"] = datafile2["Pair"].apply(lambda i: editdistance.eval(text2, i))
      datafile2 = datafile2.sort_values(by='Distance')
      vsci = list(datafile2["scientific name"])[0]
      veng = list(datafile2["English name"])[0]
      score = list(datafile2["Distance"])[0]
      if score > 5:
          print(text2)
          n = 3
          display(datafile2.head(3))
          inpu = input("Confirm?").strip()
          while inpu == 'more':
            n += 5
            display(datafile2.head(n))
            inpu = input("Confirm?").strip()
          if inpu != '':
            vsci = list(datafile2["scientific name"])[int(inpu)]
            veng = list(datafile2["English name"])[int(inpu)]
            inpu2 = input("Confirm again").strip()
            if inpu != '':
              vsci = list(datafile2["scientific name"])[int(inpu2)]
              veng = list(datafile2["English name"])[int(inpu2)]
        
  cachedsci[text] = vsci
  cachedeng[text] = veng
  clear_output()
  return (vsci, veng)

def optimum(spname, scieng):
  global done
  done += 1
  global cachedsci, cachedeng
  try:
    if scieng == 'sci':
      return cachedsci[spname.lower()]
    if scieng == 'eng':
      return cachedeng[spname.lower()]
  except KeyError:
    if scieng == 'sci': 
      return get(spname)[0]
    if scieng == 'eng': 
      return get(spname)[1]

made = make_df([i for i in spdata2 if type(i) == Sighting])
made["Comments"] = made["Species"].apply(lambda i: spdata[i].strip())

df = pd.concat([df_in, made])

def dt(row):
  d = row["Date"]
  if type(d) == str:
    try:
      if d.count('/') == 1:
        return datetime(year=2005,month=int(d.split("/")[1]),day=int(d.split("/")[0]))
      elif d.count('/') == 2:
        return datetime(year=int(d.split("/")[2]),month=int(d.split("/")[0]),day=int(d.split("/")[1]))
      if d.count('-') == 2:
        return datetime(year=int(d.split("-")[0]),month=int(d.split("-")[1]),day=int(d.split("-")[2]))
    except:
      return None
  else:
    return d

df["Date"] = df.apply(dt,axis=1)

datafile["Pair"] = datafile["Pair"].apply(lambda i: i.lower())
df["English name"] = df["Species"].progress_apply(lambda i: optimum(i, 'eng'))
df["Scientific name"] = df["Species"].apply(lambda i: optimum(i, 'sci'))
df = df[~(df["Date"]==None)]

a_file = open("cachedsci.pkl", "wb")
pickle.dump(cachedsci, a_file)
a_file.close()

a_file2 = open("cachedeng.pkl", "wb")
pickle.dump(cachedeng, a_file2)
a_file2.close()

df["Count"] = df["Count"].apply(lambda i: i if type(i) == int or str(i).isdigit() else 'X')

df = df.drop_duplicates()

cols = ["English name", "Scientific name", "Date", "Count", "Observer", "Location", "Comments"]
cols2 = ["Species", "Date", "Count", "Observer", "Location", "Comments"]

df_display = df[cols]

df_in = df.copy(deep=True)
df[cols2].to_csv("df (10).csv", index=False)
files.download("df (10).csv")

df_display.to_csv("output.csv", index=False)
files.download("output.csv")

data_table.DataTable(df_display, include_index=False)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

,English name,Scientific name,Date,Count,Observer,Location,Comments
0,Yellow-crested Cockatoo,Cacatua sulphurea,2005-01-30,1,(NK/LKS/FR/IR/JR),Malcolm Park,1 seen peeling off small branches of an albizi...
1,Tanimbar Corella,Cacatua goffiniana,2005-01-02,4,(DA/NK/LKS/JR),Changi Beach Park,"4 seen at Changi Beach Park, 2/1 (DA/NK/LKS/JR..."
2,Tanimbar Corella,Cacatua goffiniana,2005-02-27,3,(HHC/JL),Holland Woods,"4 seen at Changi Beach Park, 2/1 (DA/NK/LKS/JR..."
3,Blue-crowned Hanging-Parrot,Loriculus galgulus,2005-01-17,1,(LKS),Dairy Farm Road,"1 over Dairy Farm Road, 17/1 (LKS) and 18/1 (L..."
4,Blue-crowned Hanging-Parrot,Loriculus galgulus,2005-01-18,1,(LKS),Dairy Farm Road,"1 over Dairy Farm Road, 17/1 (LKS) and 18/1 (L..."
...,...,...,...,...,...,...,...
536,Chestnut-rumped Babbler,Stachyris maculata,2005-12-01,6,A flock of about 6 seen at Bunker Track,None,"A flock of about 6 seen at Bunker Track, 1/12 ..."
537,Chestnut-rumped Babbler,Stachyris maculata,2005-12-18,270,. A few heard at Track 270,None,"A flock of about 6 seen at Bunker Track, 1/12 ..."
538,Scarlet-breasted Flowerpecker,Prionochilus thoracicus,2005-12-01,1,1 male seen at Bunker Track,None,"1 male seen at Bunker Track, 1/12 (YDL)."
539,Plain Flowerpecker,Dicaeum minullum,2005-12-18,1,1 seen at Track 270,None,"1 seen at Track 270, 18/12 (AC/AVC/LKK/TPL/TJL..."
